# Develop improved cut flow

In [2]:
import pickle
import awkward as ak
from pyutils.pyprint import Print
from pyutils.pylogger import Logger
from pyutils.pyselect import Select
from pyutils.pycut import CutManager

printer = Print()
selector = Select()

logger = Logger()

[pyprint] ⭐️ Initialised Print with verbose = False and precision = 1


In [3]:
def _append_array(data, arr, name):
    """Helper to append arrays/masks to dev field 
    This is useful for debugging and development
    Must be trk or trkfit level
    """
    try:
        if "dev" not in ak.fields(data):
            # Initialise dev field - structure will be determined by first array added
            data = ak.with_field(data, ak.zip({name: arr}, depth_limit=1), "dev")
        else: 
            # Add new field to existing 'dev' record
            new_dev = ak.with_field(data.dev, arr, name)
            data = ak.with_field(data, new_dev, "dev")
        return data
    except Exception as e:
        print("Error appending '{name}' to data array: {e}")
        raise e

In [4]:
def load(file_name):
    try:
        with open(file_name, "rb") as f:
            return pickle.load(f)
    except Exception as e:
        print(f"[ERROR] {e}")

cry_data = load("/home/mu2ecrv/sgrant-ana/misc-ana/results_cry.pkl")
# ce_data = load("/home/mu2ecrv/sgrant-ana/misc-ana/results_signal.pkl")

In [9]:
printer.print_n_events(cry_data["trk"], 1)

[pyprint] ⭐️ Printing 1 event(s)...

-------------------------------------------------------------------------------------
trk.pdg: [11, 11, -11, -11, 11, 13, -13, 13, -13, -13, -13]
trkqual.result: [0.793, 0.84, 0.452, 0.0936, 0.0125, ..., 0.0127, 0.148, 0.0586, 0.102, 0.0143]
-------------------------------------------------------------------------------------



In [11]:
Print(True).print_n_events(cry_data["trkmc"], 1)

[pyprint] ⭐️ Initialised Print with verbose = True and precision = 1
[pyprint] ⭐️ Printing 1 event(s)...

-------------------------------------------------------------------------------------
trkmcsim.valid: [[True, True, True, True, True], [True, True, True, True, True, True], [True, True, True, True, True, True], [True, True, True, True, True], [True, True, True, True, True, True], [True, True, True, True, True], [True, True, True, True, True], [True, True, True, True, True, True], [True, True, True, True, True, True], [True, True, True, True, True], [True, True, True, True, True]]
trkmcsim.id: [[93, 0, 94, 92, 0], [92, 0, 94, 101, 93, 0], [92, 0, 94, 101, 93, 0], [93, 0, 94, 92, 0], [93, 0, 92, 94, 101, 0], [93, 0, 92, 94, 0], [92, 0, 94, 93, 0], [92, 0, 101, 94, 93, 0], [92, 0, 94, 101, 93, 0], [93, 0, 94, 92, 0], [93, 0, 92, 94, 0]]
trkmcsim.nhits: [[67, -1, 2, 2, -1], [62, -1, 14, 1, 7, -1], [64, -1, 6, 1, 2, -1], [67, -1, 2, 2, -1], [14, -1, 20, 4, 1, -1], [67, -1, 2, 2, -1], [2

In [6]:
cut = CutManager()

# Truth PDG electrons
# Best fit is an electron 
# One quality downstream track
# No quality upstream tracks 
# Select electrons

def define_cuts(data):
    """
    Test different versions of the downstream & 
    """
        ###################################################
        # Track "PID" from truth track parents 
        ###################################################
        try:
            # Truth track parent is electron 
            is_truth_electron = data["trkmc"]["trkmcsim"]["pdg"] == 11
            is_trk_parent = data["trkmc"]["trkmcsim"]["nhits"] == ak.max(data["trkmc"]["trkmcsim"]["nhits"], axis=-1)
            is_trk_parent_electron = is_truth_electron & is_trk_parent 
            has_trk_parent_electron = ak.any(is_trk_parent_electron, axis=-1) 
            # Add cut 
            cut.add_cut(
                name="truth_pid", 
                description="Track parents are electrons (truth PID)", 
                mask=has_trk_parent_electron,
                group="Preselect",
                active=True
            )
            # Append for debugging
            data = _append_array(data, has_trk_parent_electron, "has_trk_parent_electron")

        except Exception as e:
            logger.log(f"Error defining 'is_truth_electron' cut: {e}", "error") 
            raise e 
            
        ###################################################
        # Tracks intersect tracker 
        ###################################################
        try: 
            # Track segments level definition
            at_trk_front = selector.select_surface(data["trkfit"], surface_name="TT_Front") 
            at_trk_mid = selector.select_surface(data["trkfit"], surface_name="TT_Mid")
            at_trk_back = selector.select_surface(data["trkfit"], surface_name="TT_Back")
            in_trk = (at_trk_front | at_trk_mid | at_trk_back)
            
            # Track level definition 
            thru_trk = (
                ak.any(at_trk_front, axis=-1) &
                ak.any(at_trk_mid, axis=-1) &
                ak.any(at_trk_back, axis=-1) 
            )
            
            # Add cut 
            cut.add_cut(
                name="thru_trk", 
                description="Tracks intersect full tracker", 
                mask=thru_trk,
                group="Preselect",
                active=True
            )
            
            # Append for debugging
            data = _append_array(data, thru_trk, "thru_trk")
            
        except Exception as e:
            logger.log(f"Error defining 'thru_trk' cut: {e}", "error") 
            raise e
            
        ###################################################
        # Track fit hypothesis quality
        # Require good track hypotheses 
        ###################################################
        try: 
            # Track level mask
            good_trkqual = selector.select_trkqual(data["trk"], quality=0.2)
            # Add cut 
            cut.add_cut(
                name="good_trkqual",
                description=f"Track quality > 0.2",
                mask=good_trkqual,
                group="Preselect",
                active=True
            )
            # Append for debugging
            data = _append_array(data, good_trkqual, "good_trkqual")
        except Exception as e:
            logger.log(f"Error defining 'good_trkqual' cut: {e}", "error") 
            raise e

        ###################################################
        # Select electron track fit hypothesis  
        ###################################################
        try:
            # Track level definition
            is_reco_electron = selector.is_electron(data["trk"])
            # Add cut 
            cut.add_cut(
                name="is_reco_electron", 
                description="Select electron track", 
                mask=is_reco_electron,
                group="Preselect",
                active=True
            )
            # Append mask for debugging
            # data = _append_array(data, is_reco_electron, "is_reco_electron")
        except Exception as e:
            logger.log(f"Error defining 'is_reco_electron' cut: {e}", "error") 
            raise e

        ###################################################
        # Best fit is an electron
        ###################################################
        try:
            trkqual = data["trk"]["trkqual.result"] 
            pdgs = data["trk"]["trk.pdg"]
            best_trkqual_idx = ak.argmax(trkqual, axis=-1, keepdims=True)
            best_fit_pdg = ak.firsts(pdgs[ak.local_index(pdgs) == best_trkqual_idx])
            best_fit_electron = best_fit_pdg == 11
            
            # Add cut 
            cut.add_cut(
                name="best_fit_electron", 
                description="Best fit is an electron", 
                mask=best_fit_electron,
                group="Preselect",
                active=True
            )
            # Append mask for debugging
            data = _append_array(data, best_trkqual_idx, "best_trkqual_idx")
            data = _append_array(data, best_fit_pdg, "best_fit_pdg")
            data = _append_array(data, best_fit_electron, "best_fit_electron")
        except Exception as e:
            logger.log(f"Error defining 'best_fit_electron' cut: {e}", "error") 
            raise e

        ###################################################
        # One quality downstream track 
        ###################################################
        try: 
            # Trajectory is signed by charge
            pdgs = data["trk"]["trk.pdg"]
            q = -1 * data["trk"]["trk.pdg"]/abs(data["trk"]["trk.pdg"])
            p_z = data["trkfit"]["trksegs"]["mom"]["fCoordinates"]["fZ"]
            q_p_z = q * p_z

            downstream_seg = p_z > 0 # segs
            downstream_trk = ak.all(~in_trk | downstream_seg, axis=-1) # hypos
            # downstream_qual_trk = ak.all(~good_trkqual | downstream_trk, axis=-1)
            
            # Add cut 
            cut.add_cut(
                name="downstream_trk",
                description=f"Quality tracks are downstream (p_z > 0 thru tracker)",
                mask=downstream_trk,
                group="Preselect",
                active=True
            )
            # Append for debugging
            data = _append_array(data, downstream_trk, "downstream_trk")
            
        except Exception as e:
            logger.log(f"Error defining 'downstream_trk' cut: {e}", "error") 
            raise e

        return data

In [13]:
# cut = CutManager()
# def define_cuts_nominal(data):
#         ###################################################
#         # Track "PID" from truth track parents 
#         ###################################################
#         try:
#             # Truth track parent is electron 
#             is_truth_electron = data["trkmc"]["trkmcsim"]["pdg"] == 11
#             is_trk_parent = data["trkmc"]["trkmcsim"]["nhits"] == ak.max(data["trkmc"]["trkmcsim"]["nhits"], axis=-1)
#             is_trk_parent_electron = is_truth_electron & is_trk_parent 
#             has_trk_parent_electron = ak.any(is_trk_parent_electron, axis=-1) 
#             # Add cut 
#             cut.add_cut(
#                 name="truth_pid", 
#                 description="Track parents are electrons (truth PID)", 
#                 mask=has_trk_parent_electron,
#                 group="Preselect",
#                 active=True
#             )
#             # Append for debugging
#             data = _append_array(data, has_trk_parent_electron, "has_trk_parent_electron")

#         except Exception as e:
#             logger.log(f"Error defining 'is_truth_electron' cut: {e}", "error") 
#             raise e 
            
#         ###################################################
#         # Tracks intersect tracker 
#         ###################################################
#         try: 
#             # Track segments level definition
#             at_trk_front = selector.select_surface(data["trkfit"], surface_name="TT_Front") 
#             at_trk_mid = selector.select_surface(data["trkfit"], surface_name="TT_Mid")
#             at_trk_back = selector.select_surface(data["trkfit"], surface_name="TT_Back")
#             in_trk = (at_trk_front | at_trk_mid | at_trk_back)
            
#         except Exception as e:
#             logger.log(f"Error defining tracker surface masks: {e}", "error") 
#             raise e
            
#         try:
#             # Track level definition 
#             thru_trk = (
#                 ak.any(at_trk_front, axis=-1) &
#                 ak.any(at_trk_mid, axis=-1) &
#                 ak.any(at_trk_back, axis=-1) 
#             )
            
#             # Add cut 
#             cut.add_cut(
#                 name="thru_trk", 
#                 description="Tracks intersect full tracker", 
#                 mask=thru_trk,
#                 group="Preselect",
#                 active=True
#             )
            
#             # Append for debugging
#             data = _append_array(data, thru_trk, "thru_trk")
            
#         except Exception as e:
#             logger.log(f"Error defining 'thru_trk' cut: {e}", "error") 
#             raise e

#         ###################################################
#         # No duplicate hypotheses
#         # This implies multiple clusters of tracker hits
#         ###################################################
#         try:

#             # Count unique pdgs
#             pdgs = data["trk"]["trk.pdg"]
#             n_unique_pdgs = ak.num(ak.run_lengths(ak.sort(pdgs)), axis=-1)
#             n_total_pdgs = ak.num(pdgs, axis=-1)
#             has_duplicates = n_unique_pdgs < n_total_pdgs
#             no_duplicates = ~has_duplicates
        
#             # Add cut 
#             cut.add_cut(
#                 name="no_duplicates",
#                 description="No duplicate track hypotheses",
#                 mask=no_duplicates,
#                 group="Preselect",
#                 active=True
#             )

#             # Append for debugging
#             data = _append_array(data, no_duplicates, "no_duplicates")

#         except Exception as e:
#             logger.log(f"Error defining 'no_duplicates' cut: {e}", "error") 
#             raise e

#         ###################################################
#         # Best fit is an electron
#         ###################################################
#         try:
#             trkqual = data["trk"]["trkqual.result"] 
#             best_trkqual_idx = ak.argmax(trkqual, axis=-1, keepdims=True)
#             best_fit_pdg = ak.firsts(pdgs[ak.local_index(pdgs) == best_trkqual_idx])
#             best_fit_electron = best_fit_pdg == 11
            
#             # Add cut 
#             cut.add_cut(
#                 name="best_fit_electron", 
#                 description="Best fit is an electron", 
#                 mask=best_fit_electron,
#                 group="Preselect",
#                 active=True
#             )
#             # Append mask for debugging
#             data = _append_array(data, best_trkqual_idx, "best_trkqual_idx")
#             data = _append_array(data, best_fit_pdg, "best_fit_pdg")
#             data = _append_array(data, best_fit_electron, "best_fit_electron")
#         except Exception as e:
#             logger.log(f"Error defining 'best_fit_electron' cut: {e}", "error") 
#             raise e

#         ###################################################
#         # Select electron track fit hypothesis  
#         ###################################################
#         try:
#             # Track level definition
#             is_reco_electron = selector.is_electron(data["trk"])
#             # Add cut 
#             cut.add_cut(
#                 name="is_reco_electron", 
#                 description="Select electron track", 
#                 mask=is_reco_electron,
#                 group="Preselect",
#                 active=True
#             )
#             # Append mask for debugging
#             # data = _append_array(data, is_reco_electron, "is_reco_electron")
#         except Exception as e:
#             logger.log(f"Error defining 'is_reco_electron' cut: {e}", "error") 
#             raise e

#         ###################################################
#         # Track fit hypothesis quality
#         # Require good track hypotheses 
#         ###################################################
#         try: 
#             # Track level mask
#             good_trkqual = selector.select_trkqual(data["trk"], quality=0.2)
#             # Add cut 
#             cut.add_cut(
#                 name="good_trkqual",
#                 description=f"Track quality > 0.2",
#                 mask=good_trkqual,
#                 group="Preselect",
#                 active=True
#             )
#             # Append for debugging
#             data = _append_array(data, good_trkqual, "good_trkqual")
#         except Exception as e:
#             logger.log(f"Error defining 'good_trkqual' cut: {e}", "error") 
#             raise e

#         ###################################################
#         # Quality downstream tracks 
#         ###################################################
#         try: 
#             # Trajectory is signed by charge
#             q = -1 * data["trk"]["trk.pdg"]/abs(data["trk"]["trk.pdg"])
#             p_z = data["trkfit"]["trksegs"]["mom"]["fCoordinates"]["fZ"]
#             q_p_z = q * p_z

#             downstream_seg = q_p_z > 0 # segs
#             downstream_trk = ak.all(~in_trk | is_downstream, axis=-1) # hypos
#             downstream_qual_trk = ak.all(~good_trkqual | all_downstream, axis=-1)
            
#             # Add cut 
#             cut.add_cut(
#                 name="downstream_qual_trk",
#                 description=f"Quality tracks are downstream (p_z > 0 thru tracker)",
#                 mask=all_downstream,
#                 group="Preselect",
#                 active=True
#             )
#             # Append for debugging
#             data = _append_array(data, all_downstream, "all_downstream")
            
#         except Exception as e:
#             logger.log(f"Error defining 'all_downstream' cut: {e}", "error") 
#             raise e

#         return data

[pycut] ✅ Initialised


In [39]:
cut = CutManager()

# Truth PDG electrons
# Best fit is an electron 
# One quality downstream track
# No quality upstream tracks 
# Select electrons

def define_cuts(data):
    """
    Test different versions of the downstream & 
    """
        ###################################################
        # Track "PID" from truth track parents 
        ###################################################
        try:
            # Truth track parent is electron 
            is_truth_electron = data["trkmc"]["trkmcsim"]["pdg"] == 11
            is_trk_parent = data["trkmc"]["trkmcsim"]["nhits"] == ak.max(data["trkmc"]["trkmcsim"]["nhits"], axis=-1)
            is_trk_parent_electron = is_truth_electron & is_trk_parent 
            has_trk_parent_electron = ak.any(is_trk_parent_electron, axis=-1) 
            # Add cut 
            cut.add_cut(
                name="truth_pid", 
                description="Track parents are electrons (truth PID)", 
                mask=has_trk_parent_electron,
                group="Preselect",
                active=True
            )
            # Append for debugging
            data = _append_array(data, has_trk_parent_electron, "has_trk_parent_electron")

        except Exception as e:
            logger.log(f"Error defining 'is_truth_electron' cut: {e}", "error") 
            raise e 
            
        ###################################################
        # Tracks intersect tracker 
        ###################################################
        try: 
            # Track segments level definition
            at_trk_front = selector.select_surface(data["trkfit"], surface_name="TT_Front") 
            at_trk_mid = selector.select_surface(data["trkfit"], surface_name="TT_Mid")
            at_trk_back = selector.select_surface(data["trkfit"], surface_name="TT_Back")
            in_trk = (at_trk_front | at_trk_mid | at_trk_back)
            
            # Track level definition 
            thru_trk = (
                ak.any(at_trk_front, axis=-1) &
                ak.any(at_trk_mid, axis=-1) &
                ak.any(at_trk_back, axis=-1) 
            )
            
            # Add cut 
            cut.add_cut(
                name="thru_trk", 
                description="Tracks intersect full tracker", 
                mask=thru_trk,
                group="Preselect",
                active=True
            )
            
            # Append for debugging
            data = _append_array(data, thru_trk, "thru_trk")
            
        except Exception as e:
            logger.log(f"Error defining 'thru_trk' cut: {e}", "error") 
            raise e
            
        ###################################################
        # Track fit hypothesis quality
        # Require good track hypotheses 
        ###################################################
        try: 
            # Track level mask
            good_trkqual = selector.select_trkqual(data["trk"], quality=0.2)
            # Add cut 
            cut.add_cut(
                name="good_trkqual",
                description=f"Track quality > 0.2",
                mask=good_trkqual,
                group="Preselect",
                active=True
            )
            # Append for debugging
            data = _append_array(data, good_trkqual, "good_trkqual")
        except Exception as e:
            logger.log(f"Error defining 'good_trkqual' cut: {e}", "error") 
            raise e

        ###################################################
        # Select electron track fit hypothesis  
        ###################################################
        try:
            # Track level definition
            is_reco_electron = selector.is_electron(data["trk"])
            # Add cut 
            cut.add_cut(
                name="is_reco_electron", 
                description="Select electron track", 
                mask=is_reco_electron,
                group="Preselect",
                active=True
            )
            # Append mask for debugging
            # data = _append_array(data, is_reco_electron, "is_reco_electron")
        except Exception as e:
            logger.log(f"Error defining 'is_reco_electron' cut: {e}", "error") 
            raise e

        ###################################################
        # Best fit is an electron
        ###################################################
        try:
            trkqual = data["trk"]["trkqual.result"] 
            pdgs = data["trk"]["trk.pdg"]
            best_trkqual_idx = ak.argmax(trkqual, axis=-1, keepdims=True)
            best_fit_pdg = ak.firsts(pdgs[ak.local_index(pdgs) == best_trkqual_idx])
            best_fit_electron = best_fit_pdg == 11
            
            # Add cut 
            cut.add_cut(
                name="best_fit_electron", 
                description="Best fit is an electron", 
                mask=best_fit_electron,
                group="Preselect",
                active=True
            )
            # Append mask for debugging
            data = _append_array(data, best_trkqual_idx, "best_trkqual_idx")
            data = _append_array(data, best_fit_pdg, "best_fit_pdg")
            data = _append_array(data, best_fit_electron, "best_fit_electron")
        except Exception as e:
            logger.log(f"Error defining 'best_fit_electron' cut: {e}", "error") 
            raise e

        ###################################################
        # One quality downstream track 
        ###################################################
        try: 
            # Trajectory is signed by charge
            pdgs = data["trk"]["trk.pdg"]
            q = -1 * data["trk"]["trk.pdg"]/abs(data["trk"]["trk.pdg"])
            p_z = data["trkfit"]["trksegs"]["mom"]["fCoordinates"]["fZ"]
            q_p_z = q * p_z

            downstream_seg = p_z > 0 # segs
            downstream_trk = ak.all(~in_trk | downstream_seg, axis=-1) # hypos
            # downstream_qual_trk = ak.all(~good_trkqual | downstream_trk, axis=-1)
            
            # Add cut 
            cut.add_cut(
                name="downstream_trk",
                description=f"Quality tracks are downstream (p_z > 0 thru tracker)",
                mask=downstream_trk,
                group="Preselect",
                active=True
            )
            # Append for debugging
            data = _append_array(data, downstream_trk, "downstream_trk")
            
        except Exception as e:
            logger.log(f"Error defining 'downstream_trk' cut: {e}", "error") 
            raise e

        return data

[pycut] ✅ Initialised


In [40]:
cry_data_nom = define_cuts_nominal(cry_data)
#ce_data_nom = define_cuts_nominal(ce_data)

[pycut] ⭐️ Added cut truth_pid with index 0 in group 'Preselect'
[pyselect] ✅ Returning mask for trksegs with sid = 0
[pyselect] ✅ Returning mask for trksegs with sid = 1
[pyselect] ✅ Returning mask for trksegs with sid = 2
[pycut] ⭐️ Added cut thru_trk with index 1 in group 'Preselect'
[pyselect] ✅ Returning mask for trkqual > 0.2
[pycut] ⭐️ Added cut good_trkqual with index 2 in group 'Preselect'
[pyselect] ✅ Returning mask for e- tracks
[pycut] ⭐️ Added cut is_reco_electron with index 3 in group 'Preselect'
[pycut] ⭐️ Added cut best_fit_electron with index 4 in group 'Preselect'
[pycut] ⭐️ Added cut downstream_trk with index 5 in group 'Preselect'


In [41]:
def df_cut_flow(data):
    cut_flow = cut.create_cut_flow(data)
    df_cut_flow = cut.format_cut_flow(cut_flow)
    display(df_cut_flow)
    return df_cut_flow

df_cry_nom_cf = df_cut_flow(cry_data_nom)

,Cut,Group,Events Passing,Absolute [%],Relative [%],Description
0,No cuts,N/A,2610933,100.00,100.00,No selection applied
1,truth_pid,Preselect,622896,23.86,23.86,Track parents are electrons (truth PID)
2,thru_trk,Preselect,619249,23.72,99.41,Tracks intersect full tracker
3,good_trkqual,Preselect,484637,18.56,78.26,Track quality > 0.2
4,is_reco_electron,Preselect,474927,18.19,98.00,Select electron track
5,best_fit_electron,Preselect,432878,16.58,91.15,Best fit is an electron
6,downstream_trk,Preselect,324634,12.43,74.99,Quality tracks are downstream (p_z > 0 thru tr...


In [42]:
printer.print_n_events(cry_data_nom["trk"])
printer.print_n_events(cry_data_nom["dev"])

[pyprint] ⭐️ Printing 1 event(s)...

-------------------------------------------------------------------------------------
trk.pdg: [11, 11, -11, -11, 11, 13, -13, 13, -13, -13, -13]
trkqual.result: [0.793, 0.84, 0.452, 0.0936, 0.0125, ..., 0.0127, 0.148, 0.0586, 0.102, 0.0143]
-------------------------------------------------------------------------------------

[pyprint] ⭐️ Printing 1 event(s)...

-------------------------------------------------------------------------------------
has_trk_parent_electron: [True, True, True, True, True, True, True, True, True, True, True]
thru_trk: [True, True, True, True, True, True, True, True, True, True, True]
good_trkqual: [True, True, True, False, False, True, False, False, False, False, False]
best_trkqual_idx: [1]
best_fit_pdg: 11
best_fit_electron: True
downstream_trk: [True, True, False, False, False, True, True, True, False, False, False]
-------------------------------------------------------------------------------------

